# Decision Trees
In this notebook, we'll implement the decision tree algorithm, and use it in the example of identifying edible mushrooms as in the course graded lab.  
The code here are based on my own implementations in the graded lab, organized and rewritten to be more succinct and clear.

## Tools

In [182]:
import numpy as np

## Dataset

The dataset for this task is as follows:

|                                                     | Cap Color | Stalk Shape | Solitary | Edible |
|:---------------------------------------------------:|:---------:|:-----------:|:--------:|:------:|
| <img src="images/0.png" alt="drawing" width="50"/> |   Brown   |   Tapering  |    Yes   |    1   |
| <img src="images/1.png" alt="drawing" width="50"/> |   Brown   |  Enlarging  |    Yes   |    1   |
| <img src="images/2.png" alt="drawing" width="50"/> |   Brown   |  Enlarging  |    No    |    0   |
| <img src="images/3.png" alt="drawing" width="50"/> |   Brown   |  Enlarging  |    No    |    0   |
| <img src="images/4.png" alt="drawing" width="50"/> |   Brown   |   Tapering  |    Yes   |    1   |
| <img src="images/5.png" alt="drawing" width="50"/> |    Red    |   Tapering  |    Yes   |    0   |
| <img src="images/6.png" alt="drawing" width="50"/> |    Red    |  Enlarging  |    No    |    0   |
| <img src="images/7.png" alt="drawing" width="50"/> |   Brown   |  Enlarging  |    Yes   |    1   |
| <img src="images/8.png" alt="drawing" width="50"/> |    Red    |   Tapering  |    No    |    1   |
| <img src="images/9.png" alt="drawing" width="50"/> |   Brown   |  Enlarging  |    No    |    0   |


-  You have 10 examples of mushrooms. For each example, you have
    - Three features
        - Cap Color (`Brown` or `Red`),
        - Stalk Shape (`Tapering (as in \/)` or `Enlarging (as in /\)`), and
        - Solitary (`Yes` or `No`)
    - Label
        - Edible (`1` indicating yes or `0` indicating poisonous)

### One hot encoded dataset
We can one-hot encoded the features in our dataset:  

- `X_train` contains three features for each example 
    - Brown Color (A value of `1` indicates "Brown" cap color and `0` indicates "Red" cap color)
    - Tapering Shape (A value of `1` indicates "Tapering Stalk Shape" and `0` indicates "Enlarging" stalk shape)
    - Solitary  (A value of `1` indicates "Yes" and `0` indicates "No")
- `y_train` is whether the mushroom is edible 
    - `y = 1` indicates edible
    - `y = 0` indicates poisonous

In [183]:
X_train = np.array([[1,1,1],[1,0,1],[1,0,0],[1,0,0],[1,1,1],[0,1,1],[0,0,0],[1,0,1],[0,1,0],[1,0,0]])
y_train = np.array([1,1,0,0,1,0,0,1,1,0])

In [184]:
print ('The shape of X_train is:', X_train.shape)
print ('The shape of y_train is: ', y_train.shape)
print ('Number of training examples (m):', len(X_train))

The shape of X_train is: (10, 3)
The shape of y_train is:  (10,)
Number of training examples (m): 10


## Implementation
We'll follow the guidelines of the decision tree algorithm to implement our decision tree.
- Start with all examples at the root node
- Calculate information gain for all possible features, and pick one with the hightest information gain
- Split dataset according to selected feature, and create left and right branches of the tree
- Keep repeating splitting process until stopping criteria is met, here we'll use these two criterias:
    - When a node is 100% one class
    - When splitting a node will result in the tree exceeding maximum depth

Our implementation of the decision tree will consist of the following functions:
- `compute_entropy`: computes entropy at a given node
- `split_dataset`: takes in the data at a node and a feature to split on and splits it into left and right branches
- `compute_information_gain`: computes information gain of a split
- `get_best_feature`: get the best feature to split the node
- `build_tree_recursive`: builds the decision tree recursively

In [185]:
def compute_entropy(y):
    """
    Computes the entropy at a given node
    
    Args:
       y (ndarray): labels of the examples at the node
       
    Returns:
        entropy (float): Entropy at that node       
    """
    if len(y) == 0:  # If there's no example in the node, entropy defined to be 0
        return 0
        
    p1 = np.sum(y) / len(y)
    
    if p1 == 0 or p1 == 1:  # If p1 = 0 or p0 = 0, entropy is defined to be 0
        return 0

    entropy = - p1 * np.log2(p1) - (1 - p1) * np.log2(1 - p1)
    
    return entropy

In [186]:
# Compute entropy at the root node (i.e. with all examples)
# Since we have 5 edible and 5 non-edible mushrooms, the entropy should be 1"

print("Entropy at root node: ", compute_entropy(y_train)) 

Entropy at root node:  1.0


**Expected Output**:
<table>
  <tr>
    <td> <b>Entropy at root node:<b> 1.0 </td> 
  </tr>
</table>

In [187]:
def split_dataset(X, node_indices, feature):
    """
    Splits the data at the given node into left and right branches
    
    Args:
        X (ndarray):             Data matrix of shape(n_samples, n_features)
        node_indices (list):     List containing the active indices. I.e, the samples being considered at this step.
        feature (int):           Index of feature to split on
    
    Returns:
        left_indices (list):     Indices with feature value == 1
        right_indices (list):    Indices with feature value == 0
    """
    msk_arr = X[node_indices, feature] == 1  # Create a mask array for samples with feature value == 1
    left_indices = node_indices[msk_arr]
    right_indices = node_indices[~msk_arr]
    
    return left_indices, right_indices

In [188]:
# Case 1
root_indices = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
feature = 0

left_indices, right_indices = split_dataset(X_train, root_indices, feature)

print("CASE 1:")
print("Left indices: ", left_indices)
print("Right indices: ", right_indices)

# Case 2
root_indices_subset = np.array([0, 2, 4, 6, 8])
left_indices, right_indices = split_dataset(X_train, root_indices_subset, feature)

print("CASE 2:")
print("Left indices: ", left_indices)
print("Right indices: ", right_indices)

CASE 1:
Left indices:  [0 1 2 3 4 7 9]
Right indices:  [5 6 8]
CASE 2:
Left indices:  [0 2 4]
Right indices:  [6 8]


**Expected Output**:
```
CASE 1:
Left indices:  [0, 1, 2, 3, 4, 7, 9]
Right indices:  [5, 6, 8]

CASE 2:
Left indices:  [0, 2, 4]
Right indices:  [6, 8]
```

In [189]:
def compute_information_gain(X, y, node_indices, feature): 
    """
    Compute the information gain of splitting the node on a given feature
    
    Args:
        X (ndarray):            Data matrix of shape(n_samples, n_features)
        y (array like):         List or ndarray with n_samples containing the target variable
        node_indices (ndarray): List containing the active indices. I.e, the samples being considered in this step.
        feature (int):          Index of feature to split on
   
    Returns:
        info_gain (float):      Information gain
    """
    left_indices, right_indices = split_dataset(X, node_indices, feature)
    w_left = len(left_indices) / len(node_indices)
    w_right = 1 - w_left
    info_gain = compute_entropy(y[node_indices]) - (w_left * compute_entropy(y[left_indices]) + 
                                                    w_right * compute_entropy(y[right_indices]))
    return info_gain

In [190]:
info_gain0 = compute_information_gain(X_train, y_train, root_indices, feature=0)
print("Information Gain from splitting the root on brown cap: ", info_gain0)

info_gain1 = compute_information_gain(X_train, y_train, root_indices, feature=1)
print("Information Gain from splitting the root on tapering stalk shape: ", info_gain1)

info_gain2 = compute_information_gain(X_train, y_train, root_indices, feature=2)
print("Information Gain from splitting the root on solitary: ", info_gain2)

Information Gain from splitting the root on brown cap:  0.034851554559677034
Information Gain from splitting the root on tapering stalk shape:  0.12451124978365313
Information Gain from splitting the root on solitary:  0.2780719051126377


**Expected Output**:
```
Information Gain from splitting the root on brown cap:  0.034851554559677034
Information Gain from splitting the root on tapering stalk shape:  0.12451124978365313
Information Gain from splitting the root on solitary:  0.2780719051126377
```

In [191]:
def get_best_feature(X, y, node_indices):   
    """
    Returns the optimal feature and threshold value
    to split the node data 
    
    Args:
        X (ndarray):            Data matrix of shape(n_samples, n_features)
        y (array like):         list or ndarray with n_samples containing the target variable
        node_indices (ndarray): List containing the active indices. I.e, the samples being considered in this step.

    Returns:
        best_feature (int):     The index of the best feature to split
    """    
    best_feature = -1
    root_entropy = compute_entropy(y[node_indices])
    
    if root_entropy == 0:
        return best_feature

    # Vectorize the function compute_information_gain over the feature argument
    vectorized_func = np.vectorize(lambda feature: compute_information_gain(X, y, node_indices, feature))

    features = np.arange(X.shape[1])
    info_gains = vectorized_func(features)
    best_feature = np.argmax(info_gains)

    return best_feature

In [192]:
best_feature = get_best_feature(X_train, y_train, root_indices)
print("Best feature to split on: %d" % best_feature)

Best feature to split on: 2


Expect that the best feature to split on at the root node is feature 2 ("Solitary")

### Recursive function to build the decision tree
Here we implement by ourselves the recursive function to build the tree. If the one of the two following criteria is reached, stop splitting the tree and print the leaf nodes:
- When a node is 100% one class
- When splitting a node will result in the tree exceeding maximum depth

We print the tree with some formatting to make the tree structure more clear.

In [197]:
def build_tree_recursive(X, y, node_indices, branch_name, max_depth, current_depth, feature_names, indent=""):
    """
    Build a tree using the recursive algorithm that split the dataset into 2 subgroups at each node.
    This function just prints the tree.
    
    Args:
        X (ndarray):            Data matrix of shape(n_samples, n_features)
        y (array like):         list or ndarray with n_samples containing the target variable
        node_indices (ndarray): List containing the active indices. I.e, the samples being considered in this step.
        branch_name (string):   Name of the branch. ['Root', 'Left', 'Right']
        max_depth (int):        Max depth of the resulting tree. 
        current_depth (int):    Current depth. Parameter used during recursive call.
        dash (string):        Space indented for printing the tree
        feature_names (list):   List of names of the features
        
    """ 
    # Print the current branch and depth
    print("{}Depth {}, {}:".format(indent, current_depth, branch_name))
    
    # First check if the tree has reached maximum depth
    if current_depth == max_depth:
        # Reached maximum depth, stop splitting
        # Print the leaf node
        print("{}  [Leaf] Reached maximum depth. Samples: {}".format(indent, node_indices))
        return None

    # Then check if the node is at 100% purity
    best_feature = get_best_feature(X, y, node_indices)
    if best_feature == -1:
        # Node is at 100% purity, stop splitting
        # Print the leaf node
        print("{}  [Leaf] Node is at 100% purity. Samples: {}".format(indent, node_indices))
        return None

    print("{}  Split on feature {} [{}]".format(indent, best_feature, feature_names[best_feature]))
    left_indices, right_indices = split_dataset(X, node_indices, best_feature)
          
    build_tree_recursive(X, y, left_indices, "Left", max_depth, current_depth + 1, feature_names, indent + "  ")
    build_tree_recursive(X, y, right_indices, "Right", max_depth, current_depth + 1, feature_names, indent + "  ")

In [198]:
root_indices = np.arange(X_train.shape[0])
feature_names = ["Brown cap color", "Tapering Stalk Shape", "Solitary"]

In [199]:
build_tree_recursive(X_train, y_train, root_indices, "Root", 2, 0, feature_names, "")

Depth 0, Root:
  Split on feature 2 [Solitary]
  Depth 1, Left:
    Split on feature 0 [Brown cap color]
    Depth 2, Left:
      [Leaf] Reached maximum depth. Samples: [0 1 4 7]
    Depth 2, Right:
      [Leaf] Reached maximum depth. Samples: [5]
  Depth 1, Right:
    Split on feature 1 [Tapering Stalk Shape]
    Depth 2, Left:
      [Leaf] Reached maximum depth. Samples: [8]
    Depth 2, Right:
      [Leaf] Reached maximum depth. Samples: [2 3 6 9]


In [200]:
build_tree_recursive(X_train, y_train, root_indices, "Root", 4, 0, feature_names, "")

Depth 0, Root:
  Split on feature 2 [Solitary]
  Depth 1, Left:
    Split on feature 0 [Brown cap color]
    Depth 2, Left:
      [Leaf] Node is at 100% purity. Samples: [0 1 4 7]
    Depth 2, Right:
      [Leaf] Node is at 100% purity. Samples: [5]
  Depth 1, Right:
    Split on feature 1 [Tapering Stalk Shape]
    Depth 2, Left:
      [Leaf] Node is at 100% purity. Samples: [8]
    Depth 2, Right:
      [Leaf] Node is at 100% purity. Samples: [2 3 6 9]


We can see that when we set the maximum depth to 2, the tree stops splitting because the maximum depth is reached; when we set the maximum depth to 4, the tree stops splitting because the nodes are at 100% purity. The splitting result is consistent with the result we got in the lab.